<a href="https://colab.research.google.com/github/Np2525/BudgetWise-Model/blob/main/BudgetWiseFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Enable GPU acceleration (Colab)
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

# ✅ 1️⃣ Load Food-101 Dataset (Without Downloading)
dataset_name = "food101"
(train_data, val_data), dataset_info = tfds.load(
    dataset_name, split=["train[:10%]", "validation[:10%]"],  # Using only 10% of data to prevent crashes
    as_supervised=True, with_info=True
)

# ✅ 2️⃣ Reduce Memory Usage (Batching & Resizing)
BATCH_SIZE = 16  # Small batch size to save RAM
IMG_SIZE = (160, 160)  # Reduce image size to lower memory use

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE) / 255.0  # Normalize pixels
    return image, label

train_data = train_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_data = val_data.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ✅ 3️⃣ Use MobileNetV2 (Optimized for Low Memory)
base_model = MobileNetV2(input_shape=(160, 160, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base layers

# Build Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(101, activation='softmax')  # 101 food categories
])

# ✅ 4️⃣ Compile Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ✅ 5️⃣ Train in Small Epochs (Prevent Crashes)
history = model.fit(train_data, validation_data=val_data, epochs=3)  # Start with 3 epochs

# ✅ 6️⃣ Save Model (Optional)
model.save("food101_mobilenetv2.h5")
print("Model saved!")

# ✅ 7️⃣ Test on a New Image (Food Classification)
def predict_food(image_path):
    img = load_img(image_path, target_size=(160, 160))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)

    class_names = dataset_info.features["label"].names
    print("Predicted Food:", class_names[predicted_class])

# Example: Run prediction (upload a food image in Colab)
# predict_food("your_image.jpg")


Num GPUs Available: 0
Epoch 1/3
474/474 ━━━━━━━━━━━━━━━━━━━━ 258s 531ms/step - accuracy: 0.1107 - loss: 4.1388 - val_accuracy: 0.3442 - val_loss: 2.6811
Epoch 2/3
474/474 ━━━━━━━━━━━━━━━━━━━━ 264s 557ms/step - accuracy: 0.4215 - loss: 2.3313 - val_accuracy: 0.3941 - val_loss: 2.4364
Epoch 3/3
474/474 ━━━━━━━━━━━━━━━━━━━━ 260s 548ms/step - accuracy: 0.5456 - loss: 1.7717 - val_accuracy: 0.4099 - val_loss: 2.3912


Model saved!


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# ✅ 1️⃣ Load the trained model
model = tf.keras.models.load_model("food101_mobilenetv2.h5")
print("Model loaded successfully!")

# ✅ 2️⃣ Load Food-101 class names
import tensorflow_datasets as tfds
dataset_info = tfds.builder("food101").info
class_names = dataset_info.features["label"].names  # Get food categories

# ✅ 3️⃣ Upload an image (Run this in Colab)
from google.colab import files
uploaded = files.upload()  # Upload a food image (e.g., "my_food.jpg")

image_path = list(uploaded.keys())[0]  # Get uploaded image filename

# ✅ 4️⃣ Preprocess Image
IMG_SIZE = (160, 160)
img = load_img(image_path, target_size=IMG_SIZE)  # Resize image
img_array = img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions

# ✅ 5️⃣ Make a Prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

# ✅ 6️⃣ Print Predicted Food Name
print(f"Predicted Food: {class_names[predicted_class]}")


Model loaded successfully!


Saving food1.jpeg to food1.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Food: panna_cotta
